# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments           Customer  \
0  Sep 16 2022 10:10AM  246602        12       U1026   Uniderm USA, Inc   
1  Sep 16 2022 10:10AM  246602        12       U1024   Uniderm USA, Inc   
2  Sep 16 2022 10:10AM  246602        12       U1025   Uniderm USA, Inc   
3  Sep 16 2022 10:10AM  246601        10  0085985556  ISDIN Corporation   
4  Sep 16 2022 10:10AM  246601        10  0085985581  ISDIN Corporation   
5  Sep 16 2022 10:10AM  246601        10  0085985580  ISDIN Corporation   
6  Sep 16 2022 10:10AM  246601        10  0085985590  ISDIN Corporation   
7  Sep 16 2022 10:10AM  246601        10  0085985591  ISDIN Corporation   
8  Sep 16 2022 10:10AM  246601        10  0085985608  ISDIN Corporation   
9  Sep 16 2022 10:10AM  246601        10  0085985607  ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
21  246595       Released          1
22  246597       Released          1
23  246598       Released         10
24  246601       Released          7
25  246602       Released          3

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
246595                NaN        NaN       1.0
246597                NaN        NaN       1.0
246598                NaN        NaN      10.0
246601                NaN        NaN       7.0
246602                NaN        NaN       3.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
246294                0.0        1.0       0.0
246296                0.0        1.0       0.0
246510                0.0        0.0       1.0
246517                0.0        0.0       1.0
246528                0.0        0.0       2.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
246294                  0          1         0
246296                  0          1         0
246510                  0          0         1
246517                  0          0         1
246528                  0          0         2

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               246294          0          1         0
1               246296          0          1         0
2               246510          0          0         1
3               246517          0          0         1
4               246528          0          0         2

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               246294                   1         
1               246296                   1         
2               246510                            1
3               246517                            1
4               246528                            2

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                Customer
0   Sep 16 2022 10:10AM  246602        12        Uniderm USA, Inc
3   Sep 16 2022 10:10AM  246601        10       ISDIN Corporation
10  Sep 16 2022  9:51AM  246598        10        Eywa Pharma Inc.
20  Sep 16 2022  9:49AM  246296        21       NBTY Global, Inc.
21  Sep 16 2022  9:48AM  246294        21       NBTY Global, Inc.
22  Sep 16 2022  9:42AM  246597        16           TASA USA Inc.
23  Sep 16 2022  9:33AM  246595        16           TASA USA Inc.
24  Sep 16 2022  9:28AM  246594        19  GCH Granules USA, Inc.
25  Sep 16 2022  8:36AM  246589        19  GCH Granules USA, Inc.
26  Sep 16 2022  8:36AM  246590        10        Eywa Pharma Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                Customer Completed  \
0  Sep 16 2022 10:10AM  246602        12        Uniderm USA, Inc             
1  Sep 16 2022 10:10AM  246601        10       ISDIN Corporation             
2  Sep 16 2022  9:51AM  246598        10        Eywa Pharma Inc.             
3  Sep 16 2022  9:49AM  246296        21       NBTY Global, Inc.             
4  Sep 16 2022  9:48AM  246294        21       NBTY Global, Inc.             
5  Sep 16 2022  9:42AM  246597        16           TASA USA Inc.             
6  Sep 16 2022  9:33AM  246595        16           TASA USA Inc.             
7  Sep 16 2022  9:28AM  246594        19  GCH Granules USA, Inc.             
8  Sep 16 2022  8:36AM  246589        19  GCH Granules USA, Inc.             
9  Sep 16 2022  8:36AM  246590        10        Eywa Pharma Inc.             

  Executing Released  
0                  3  
1                  7  
2                 10  
3         1           
4         1           
5                  1  
6                  1  
7                  1  
8         1           
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                Customer Released  \
0  Sep 16 2022 10:10AM  246602        12        Uniderm USA, Inc        3   
1  Sep 16 2022 10:10AM  246601        10       ISDIN Corporation        7   
2  Sep 16 2022  9:51AM  246598        10        Eywa Pharma Inc.       10   
3  Sep 16 2022  9:49AM  246296        21       NBTY Global, Inc.            
4  Sep 16 2022  9:48AM  246294        21       NBTY Global, Inc.            
5  Sep 16 2022  9:42AM  246597        16           TASA USA Inc.        1   
6  Sep 16 2022  9:33AM  246595        16           TASA USA Inc.        1   
7  Sep 16 2022  9:28AM  246594        19  GCH Granules USA, Inc.        1   
8  Sep 16 2022  8:36AM  246589        19  GCH Granules USA, Inc.            
9  Sep 16 2022  8:36AM  246590        10        Eywa Pharma Inc.        1   

  Executing Completed  
0                      
1                      
2                      
3         1            
4         1            
5                      
6                      
7                      
8         1            
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse           Customer Released  \
0  Sep 16 2022 10:10AM  246602        12   Uniderm USA, Inc        3   
1  Sep 16 2022 10:10AM  246601        10  ISDIN Corporation        7   
2  Sep 16 2022  9:51AM  246598        10   Eywa Pharma Inc.       10   
3  Sep 16 2022  9:49AM  246296        21  NBTY Global, Inc.            
4  Sep 16 2022  9:48AM  246294        21  NBTY Global, Inc.            

  Executing Completed  
0                      
1                      
2                      
3         1            
4         1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Sep 16 2022 10:10AM  246602        12   Uniderm USA, Inc       3.0   
1  Sep 16 2022 10:10AM  246601        10  ISDIN Corporation       7.0   
2  Sep 16 2022  9:51AM  246598        10   Eywa Pharma Inc.      10.0   
3  Sep 16 2022  9:49AM  246296        21  NBTY Global, Inc.       NaN   
4  Sep 16 2022  9:48AM  246294        21  NBTY Global, Inc.       NaN   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        1.0        NaN  
4        1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Sep 16 2022 10:10AM  246602        12   Uniderm USA, Inc       3.0   
1  Sep 16 2022 10:10AM  246601        10  ISDIN Corporation       7.0   
2  Sep 16 2022  9:51AM  246598        10   Eywa Pharma Inc.      10.0   
3  Sep 16 2022  9:49AM  246296        21  NBTY Global, Inc.       0.0   
4  Sep 16 2022  9:48AM  246294        21  NBTY Global, Inc.       0.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        1.0        0.0  
4        1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2465625      41.0       11.0        1.0
12          246602       3.0        0.0        0.0
16          986287       5.0        0.0        0.0
19          739767       2.0        1.0        0.0
20          246580       2.0        0.0        6.0
21          492590       0.0        2.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2465625      41.0       11.0        1.0
1        12   246602       3.0        0.0        0.0
2        16   986287       5.0        0.0        0.0
3        19   739767       2.0        1.0        0.0
4        20   246580       2.0        0.0        6.0
5        21   492590       0.0        2.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      41.0       11.0        1.0
1        12       3.0        0.0        0.0
2        16       5.0        0.0        0.0
3        19       2.0        1.0        0.0
4        20       2.0        0.0        6.0
5        21       0.0        2.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   41.0
1        12  Released    3.0
2        16  Released    5.0
3        19  Released    2.0
4        20  Released    2.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12   16   19   20   21
Status                                  
Completed   1.0  0.0  0.0  0.0  6.0  0.0
Executing  11.0  0.0  0.0  1.0  0.0  2.0
Released   41.0  3.0  5.0  2.0  2.0  0.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12   16   19   20   21
0          Completed   1.0  0.0  0.0  0.0  6.0  0.0
1          Executing  11.0  0.0  0.0  1.0  0.0  2.0
2           Released  41.0  3.0  5.0  2.0  2.0  0.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12   16   19   20   21
0  Completed   1.0  0.0  0.0  0.0  6.0  0.0
1  Executing  11.0  0.0  0.0  1.0  0.0  2.0
2   Released  41.0  3.0  5.0  2.0  2.0  0.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()